In [1]:
import CeLEry as cel

import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.model_selection import train_test_split
from anndata import AnnData, read_h5ad

import json

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_merfish_raw = pd.read_csv("data/HumanLiverCancerPatient2_cell_by_gene.csv", index_col=0)   
meta_data = pd.read_csv("data/HumanLiverCancerPatient2_cell_metadata.csv", index_col=0)
meta_data = meta_data.sort_index()

data_merfish = AnnData(data_merfish_raw)
data_merfish.obs['x_cord'] = meta_data['center_x'].tolist()
data_merfish.obs['y_cord'] = meta_data['center_y'].tolist()
data_merfish_raw = data_merfish.copy()

def findBlank(name):
    return "Blank" in name

blank_lst = np.array(list(map(findBlank, data_merfish.var.index)))
data_merfish = data_merfish[:, blank_lst == False]

sc.pp.filter_cells(data_merfish, min_genes=100)
sc.pp.filter_cells(data_merfish, min_counts=500)

sc.pp.neighbors(data_merfish, n_neighbors = 15, use_rep="X")
sc.tl.louvain(data_merfish, 0.3, random_state=1)

data_merfish.write_h5ad("data/liver_merfish.h5ad")

In [2]:
data_merfish = read_h5ad("data/liver_merfish.h5ad")

In [ ]:
Rdata_ind, Qdata_ind, _, _ =train_test_split(range(data_merfish.shape[0]), data_merfish.obs['louvain'], test_size=0.5,random_state=1,stratify=data_merfish.obs['louvain'])

Rdata = data_merfish[np.sort(Rdata_ind), :]
Qdata = data_merfish[np.sort(Qdata_ind), :]
print((Rdata.shape, Qdata.shape))

In [ ]:
cel.get_zscore(Qdata)
cel.get_zscore(Rdata)

In [10]:
Rdata.obs = Rdata.obs[['x_cord', 'y_cord']]
model_train = cel.Fit_cord (data_train = Rdata, hidden_dims = [50, 20, 10], num_epochs_max = 3000, number_error_try=50, batch_size = 128, path = "output/liver", filename = "liver_merfish")

In [ ]:
pred_cord = cel.Predict_cord (data_test = Qdata, path = "output/liver", filename = "liver_merfish")

data_train = Qdata.copy()
traindata = (data_train.X.A if issparse(data_train.X) else data_train.X)
tdatax = np.expand_dims(traindata, axis = 0)
tdata_rs = np.swapaxes(tdatax, 1, 2)
test_cord = cel.wrap_gene_location(tdata_rs, data_train.obs[['x_cord', 'y_cord']])

pred_cord_transformx = pred_cord[:,0]*(test_cord.xmax-test_cord.xmin) + test_cord.xmin
pred_cord_transformy = pred_cord[:,1]*(test_cord.ymax-test_cord.ymin) + test_cord.ymin
pred_cord_transform = np.array([pred_cord_transformx, pred_cord_transformy]).T

np.save("output/liver/celery_liver.npy", pred_cord_transform)